# Part 1: Fashion MNST 

Train various classifiers, such as a **Random Forest classifier**, an **Extra-Trees classifier**, and an
**MLP classifier** (use code mlp_clf = MLPClassifier(random_state=42)). 

Next, try to combine them into an ensemble that outperforms them all on the validation set, using a soft or hard voting classifier. Once you have found one, try it on the test set. How much better does it perform compared to the individual classifiers?

Run the individual classifiers to make predictions on the validation set, and create a new training set with the resulting predictions: each training instance is a vector containing the set of predictions from all your classifiers for an image, and the target is the image’s class. Congratulations, you have just trained a blender, and together with the classifiers they form a stacking ensemble! Now let’s evaluate the ensemble on the test set. For each image in the test set, make predictions with all your classifiers, then feed the predictions to the blender to get the ensemble’s predictions. How does it compare to the voting classifier you trained earlier?

In [1]:
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [2]:
import mnist_reader
X_train, y_train = mnist_reader.load_mnist('', kind='train')
X_test, y_test = mnist_reader.load_mnist('', kind='t10k')

In [3]:
X_train.shape

(60000, 784)

In [4]:
y_train.shape

(60000,)

In [5]:
X_test.shape

(10000, 784)

In [6]:
y_test.shape

(10000,)

We will now split our training set for a validation set

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=1/6, random_state=42)

In [8]:
X_train.shape

(50000, 784)

In [9]:
y_train.shape

(50000,)

In [10]:
X_val.shape

(10000, 784)

In [11]:
y_val.shape

(10000,)

# Define individual classifier for use

We will import and define our individual classifiers to us later for voting and stacking ensemble
- RandomForestClassifier
- ExtraTreesClassifier
- MLPClassifier

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

forest_clf = RandomForestClassifier(n_estimators=500, random_state=42)
extratrees_clf = ExtraTreesClassifier(n_estimators=500,random_state=42)
mlp_clf = MLPClassifier(random_state=42)

###  Ensemble Voting Classifier (Hard)

Now we will use the individual classifiers to train a hard voting classifier and compare with them individually

In [13]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators=[('rf', forest_clf), ('et', extratrees_clf), ('mlp', mlp_clf)],
    voting='hard')

In [14]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     n_estimators=500,
                                                     n_jobs=None,
          

In [16]:
y_pred = voting_clf.predict(X_val)
print(voting_clf.__class__.__name__, accuracy_score(y_val, y_pred))

VotingClassifier 0.8845


### Ensemble Voting Classifier (Soft)

In [17]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators=[('rf', forest_clf), ('et', extratrees_clf), ('mlp', mlp_clf)],
    voting='soft')

In [18]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     n_estimators=500,
                                                     n_jobs=None,
          

In [19]:
print(voting_clf.__class__.__name__, accuracy_score(y_val, y_pred))

VotingClassifier 0.8845


We can notice that the soft classifier is less accurate than the hard one.

So lets use the hard voting classifier to predict on the test set

In [20]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators=[('rf', forest_clf), ('et', extratrees_clf), ('mlp', mlp_clf)],
    voting='hard')

In [ ]:
from sklearn.metrics import accuracy_score

for clf in (forest_clf, extratrees_clf, mlp_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

RandomForestClassifier 0.8777
ExtraTreesClassifier 0.8768
MLPClassifier 0.8469


## Training Our Classifiers

Lets define our classifiers in a list 

In [32]:
estimators = [forest_clf, extratrees_clf, mlp_clf]

Now lets train our models on the validation set

In [33]:
for estimator in estimators:
    print("Training the", estimator)
    estimator.fit(X_train, y_train)

Training the RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
Training the ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=500,
                     n_jobs=None, oob_score=False, random_state=42, verbose=0,
                     warm_start=False)
Training 

In [34]:
for estimator in estimators:
    print(estimator.__class__.__name__, estimator.score(X_val, y_val))

RandomForestClassifier 0.8838
ExtraTreesClassifier 0.8832
MLPClassifier 0.8524


Lets now create a new training set by making predicts on our validation set using all the classifiers

In [35]:
X_val_pred = np.empty((len(X_val), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_val_pred[:, index] = estimator.predict(X_val)

Lets now create a new test set by making predicts on our test set using all the classifiers

In [36]:
X_test_pred = np.empty((len(X_test), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_test_pred[:, index] = estimator.predict(X_test)

# Stacking Ensemble

Lets make blenders for our classifiers

#### Random Forest Blender

Lets train our random forest again on the valdation set to test on the testing set

In [37]:
rnd_forest_blender = RandomForestClassifier(n_estimators=500, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_val_pred, y_val)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=True, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
rnd_forest_blender.oob_score_

#### Extra Tress Blender 

We do the same on extra trees

In [39]:
extra_trees_blender = ExtraTreesClassifier(n_estimators=500, bootstrap=True, oob_score=True, random_state=42)
extra_trees_blender.fit(X_val_pred, y_val)

ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=500,
                     n_jobs=None, oob_score=True, random_state=42, verbose=0,
                     warm_start=False)

In [40]:
extra_trees_blender.oob_score_

0.8836

#### MLP Blender 

Lastly, the same for the MLP

In [43]:
mlp_blender = MLPClassifier(max_iter=1200, random_state=42)
mlp_blender.fit(X_val_pred, y_val)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=1200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=42, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

# Evaluate Our Stacking Ensemble

Lets evaluate the Stack Ensemble using the test set

In [45]:
%matplotlib
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

for blender in (rnd_forest_blender, extra_trees_blender, mlp_blender):
    y_pred = blender.predict(X_test_pred)
    print(blender.__class__.__name__, accuracy_score(y_test, y_pred))
    
    fig, ax = plt.subplots(figsize=(15,15))
    sns.heatmap(data=confusion_matrix(y_test, y_pred), annot=True,ax=ax)
    plt.show()

RandomForestClassifier 0.8774
ExtraTreesClassifier 0.8776
MLPClassifier 0.8719


We can notice that the stacking ensemble is better than the voting classification. Using the predicted values of the classifiers on the validation data really helps out